## Import

In [2]:
import random
import pandas as pd
import numpy as np
import os
import glob

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from tqdm.auto import tqdm

import warnings
warnings.filterwarnings(action='ignore') 

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

## Hyperparameter Setting

In [149]:
CFG = {
    'EPOCHS':500,
    'LEARNING_RATE':1e-3,
    'BATCH_SIZE':128,
    'SEED':41
}

## Fixed RandomSeed

In [150]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

## Data Pre-processing

In [151]:
all_input_list = sorted(glob.glob('./data/train_input/*.csv'))
all_target_list = sorted(glob.glob('./data/train_target/*.csv'))

In [152]:
train_input_list = all_input_list[:25]
train_target_list = all_target_list[:25]

val_input_list = all_input_list[25:]
val_target_list = all_target_list[25:]

In [153]:
train = pd.read_csv('lstm_train.csv')

In [154]:
input_df = train.drop(['obs_time', 'predicted_weight_g'], axis=1)
target_df = train['predicted_weight_g']

In [155]:
test = pd.read_csv('lstm_test.csv')

## CustomDataset

In [156]:
class CustomDataset(Dataset):
    def __init__(self, input_paths, target_paths, infer_mode, train=True):
        self.input_paths = input_paths
        self.target_paths = target_paths
        self.infer_mode = infer_mode
        
        self.data_list = []
        self.label_list = []
        print('Data Pre-processing..')
        if train:    
            for idx in range(728):
                time_series = input_df[24*idx:24*(idx+1)].values
                target_value = target_df[24*idx:24*(idx+1)].values.mean()
                self.data_list.append(torch.Tensor(time_series))
                self.label_list.append(torch.from_numpy(np.array(target_value)))
        else:            
            for idx in range(728, 782):
                time_series = input_df[24*idx:24*(idx+1)].values
                target_value = target_df[24*idx:24*(idx+1)].values.mean()
                self.data_list.append(torch.Tensor(time_series))
                self.label_list.append(torch.from_numpy(np.array(target_value)))    
        print('Done.')
                      
    def __getitem__(self, idx):
        data = self.data_list[idx]
        label = self.label_list[idx]
        if self.infer_mode == False:
            return data, label
        else:
            return data
        
    def __len__(self):
        return len(self.data_list)

In [157]:
train_dataset = CustomDataset(train_input_list, train_target_list, False, True)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False)

val_dataset = CustomDataset(val_input_list, val_target_list, False, False)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False)

Data Pre-processing..
Done.
Data Pre-processing..
Done.


## Model Define

In [114]:
# class BaseModel(nn.Module):
#     def __init__(self):
#         super(BaseModel, self).__init__()
#         self.lstm = nn.LSTM(input_size=35, hidden_size=256, batch_first=True, bidirectional=False)
#         self.classifier = nn.Sequential(
#             nn.Linear(256, 1),
#         )
        
#     def forward(self, x):
#         hidden, _ = self.lstm(x)
#         output = self.classifier(hidden[:,-1,:])
#         return output

In [139]:
class Conv1d_LSTM(nn.Module):
    def __init__(self, in_channel=35, out_channel=1):
        super(Conv1d_LSTM, self).__init__()
        self.layer1 = torch.nn.Sequential(
            nn.Conv1d(in_channel, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU())
        
        self.layer2 = torch.nn.Sequential(
            nn.Conv1d(64, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU())
        
        self.layer3 = torch.nn.Sequential(
            nn.Conv1d(32, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU())
        
        self.layer4 = torch.nn.Sequential(
            nn.Conv1d(32, 16, kernel_size=4, stride=1, padding=1),
            nn.ReLU())

        self.lstm = nn.LSTM(input_size=16,
                    hidden_size=64,
                    num_layers=1,
                    bias=True,
                    bidirectional=False,
                    batch_first=True)
        
        self.dropout = nn.Dropout(0.5)
        self.linear = nn.Linear(64, 32)
        self.regressor = nn.Linear(32, out_channel)
        
    def forward(self, x):
        x = x.transpose(1, 2)
        # print(f'input_shape: {x.shape}')
        # (Batch, 3, 128, 128)
        x = self.layer1(x)
        # print(f'layer1_shape: {x.shape}')
        # (Batch, 8, 64, 64)
        x = self.layer2(x)
        # print(f'layer2_shape: {x.shape}')
        # (Batch, 16, 32, 32)
        x = self.layer3(x)
        # print(f'layer3_shape: {x.shape}')
        # (Batch, 32, 16, 16)
        x = self.layer4(x)
        # print(f'layer4_shape: {x.shape}')
        # (Batch, 64, 7, 7) -> Flatten (Batch, 64*7*7(=3136))
        x = x.transpose(1, 2)
        
        # print(f'flatten_shape: {x.shape}')
        # Regressor (Batch, 3136) -> (Batch, 1)
        self.lstm.flatten_parameters()
        _, (hidden, _) = self.lstm(x)
        x = hidden[-1]
        x = self.dropout(x)
        x = self.linear(x)
        out = self.regressor(x)

        return out

## Train

In [140]:
def validation(model, val_loader, criterion, device):
    model.eval()
    val_loss = []
    with torch.no_grad():
        for X, Y in iter(val_loader):
            X = X.float().to(device)
            Y = Y.float().to(device)
            
            model_pred = model(X)
            loss = criterion(model_pred, Y)
            # print(f'True: {Y}, \n prediction: {model_pred}')
            # print(f'validation X: {X}, \n shape: {X.shape}')
            val_loss.append(loss.item())
            
    return np.mean(val_loss)

In [161]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.L1Loss().to(device)
    
    best_loss = 9999
    best_model = None
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for X, Y in iter(train_loader):
            X = X.to(device)
            Y = Y.to(device)
            optimizer.zero_grad()
            
            output = model(X)
            loss = criterion(output, Y)
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        val_loss = validation(model, val_loader, criterion, device)
        
        print(f'Train Loss : [{np.mean(train_loss):.5f}] Valid Loss : [{val_loss:.5f}]')
        
        if scheduler is not None:
            scheduler.step(val_loss)
            
        if best_loss > val_loss:
            best_loss = val_loss
            best_model = model
    return best_model

## Run!!

In [162]:
model =  Conv1d_LSTM()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, threshold_mode='abs',min_lr=1e-3, verbose=True)

best_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

Train Loss : [35.49249] Valid Loss : [21.55723]
Train Loss : [35.07952] Valid Loss : [21.13371]
Train Loss : [34.68534] Valid Loss : [20.65334]
Train Loss : [34.25642] Valid Loss : [20.27967]
Train Loss : [34.12166] Valid Loss : [20.02550]
Train Loss : [33.67040] Valid Loss : [19.31536]
Train Loss : [33.21599] Valid Loss : [18.83912]
Train Loss : [32.85073] Valid Loss : [18.33767]
Train Loss : [32.52983] Valid Loss : [17.71935]
Train Loss : [32.15223] Valid Loss : [17.64507]
Train Loss : [31.68595] Valid Loss : [17.26768]
Train Loss : [31.31356] Valid Loss : [17.02126]
Train Loss : [31.20458] Valid Loss : [17.56978]
Train Loss : [31.42249] Valid Loss : [17.41329]
Train Loss : [31.31740] Valid Loss : [17.81461]
Train Loss : [31.41338] Valid Loss : [17.12337]
Train Loss : [30.90468] Valid Loss : [16.81198]
Train Loss : [30.52727] Valid Loss : [16.70567]
Train Loss : [30.25870] Valid Loss : [16.65165]
Train Loss : [30.07626] Valid Loss : [16.64083]
Train Loss : [29.91866] Valid Loss : [16

## Inference

In [163]:
test_input_list = sorted(glob.glob('./data/test_input/*.csv'))
test_target_list = sorted(glob.glob('./data/test_target/*.csv'))

In [164]:
class CustomDataset_test(Dataset):
    def __init__(self, infer_mode):
        self.infer_mode = infer_mode
        
        self.data_list = []
        self.val_list = []
        self.label_list = []
        print('Data Pre-processing..')


        for idx in range(140):
            time_series = input_df[24*idx:24*(idx+1)].values
            target_value = target_df[24*idx:24*(idx+1)].values.mean()
            self.data_list.append(torch.Tensor(time_series))
            self.label_list.append(torch.from_numpy(np.array(target_value)))
        print('Done.')
                      
    def __getitem__(self, idx):
        data = self.data_list[idx]
        label = self.label_list[idx]
        if self.infer_mode == False:
            return data, label
        else:
            return data
        
    def __len__(self):
        return len(self.data_list)

In [165]:
test_dataset = CustomDataset_test(True)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False)

Data Pre-processing..
Done.


In [166]:
def inference_per_case(model, test_loader, device):
    model.to(device)
    model.eval()
    pred_list = []
    with torch.no_grad():
        for X in iter(test_loader):
            X = X.float().to(device)
            
            model_pred = model(X)
            
            model_pred = model_pred.cpu().numpy().reshape(-1).tolist()
            
            pred_list += model_pred
    
    return pred_list

In [167]:
inference_per_case(best_model, test_loader, device)

[12.920406341552734,
 12.77182674407959,
 12.995802879333496,
 12.995802879333496,
 12.995802879333496,
 12.995802879333496,
 12.995802879333496,
 12.995802879333496,
 12.995802879333496,
 12.995802879333496,
 12.995802879333496,
 12.995802879333496,
 12.995802879333496,
 12.995802879333496,
 12.995802879333496,
 12.995802879333496,
 12.995802879333496,
 12.995802879333496,
 12.995802879333496,
 12.995802879333496,
 12.995802879333496,
 12.995802879333496,
 12.995802879333496,
 12.995802879333496,
 12.995802879333496,
 12.995802879333496,
 12.995802879333496,
 12.995802879333496,
 13.085265159606934,
 10.505683898925781,
 10.505683898925781,
 10.505683898925781,
 10.505683898925781,
 10.505683898925781,
 10.505683898925781,
 10.505683898925781,
 10.505683898925781,
 10.505683898925781,
 10.505683898925781,
 10.505683898925781,
 10.505683898925781,
 10.505683898925781,
 10.505683898925781,
 10.505683898925781,
 10.505683898925781,
 10.505683898925781,
 10.505683898925781,
 10.5056838989

In [168]:
for test_input_path, test_target_path in zip(test_input_list, test_target_list):
    test_dataset = CustomDataset([test_input_path], [test_target_path], True)
    test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False)
    inference_per_case(best_model, test_loader, test_target_path, device)

Data Pre-processing..
Done.


TypeError: inference_per_case() takes 3 positional arguments but 4 were given

## Submission

In [72]:
import zipfile
os.chdir("./test_target/")
submission = zipfile.ZipFile("../submission.zip", 'w')
for path in test_target_list:
    path = path.split('/')[-1]
    submission.write(path)
submission.close()

FileNotFoundError: [WinError 2] 지정된 파일을 찾을 수 없습니다: './test_target/'